In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Model, load_model
from keras.layers import Input, Dense
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Data/PCA_Normalized.csv', index_col = 0)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,BBLE
0,0.535946,-0.340776,-0.378950,-0.409478,0.385434,0.752969,-0.966496,-0.007804,1000010101
1,4.410000,15.830565,7.294441,-3.931946,1.580675,15.990311,4.177770,1.256324,1000010201
2,-0.002913,0.070488,-0.009502,0.081313,-0.012082,-0.091095,-0.047430,0.064557,1000020001
3,0.035085,-0.062495,-0.065342,0.002053,-0.036879,-0.070413,-0.107799,-0.015961,1000020023
4,7.995610,-3.547684,-2.722607,-4.061353,-10.119040,9.130893,-10.615977,-13.862914,1000030001


In [4]:
mat = df.iloc[:,:-1].values.astype(np.float32)

In [5]:
mat.shape

(1070994, 8)

In [10]:
input_dim = mat.shape[1]  # 设定输入层
encoding_dim = 4  # 设定编码层

input_layer = Input(shape=(input_dim, ))  # 添加输入层

encoder = Dense(encoding_dim, activation="tanh")(input_layer)  # 编码器输入层使用tanh激活函数

decoder = Dense(input_dim, activation='tanh')(encoder)  # 解码器输入层使用tanh激活函数

autoencoder = Model(inputs=input_layer, outputs=decoder)  # 创建自动编码模型

In [15]:
nb_epoch = 3  # 设定循环次数

autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])  # 编译自动编码器

y_predictor = autoencoder.fit(mat, mat, epochs=nb_epoch, shuffle=True)  # 训练模型

Epoch 1/3
1070994/1070994 [==============================] - 23s 21us/step - loss: 0.9643 - acc: 0.7174
Epoch 2/3
1070994/1070994 [==============================] - 22s 21us/step - loss: 0.9642 - acc: 0.7175
Epoch 3/3
1070994/1070994 [==============================] - 22s 21us/step - loss: 0.9643 - acc: 0.7177


In [16]:
predictions = autoencoder.predict(mat)

In [39]:
pd.DataFrame(predictions).head()

,0,1,2,3,4,5,6,7
0,0.512801,-0.312848,-0.146774,-0.670848,-0.027296,0.900714,-0.714181,-0.411540
1,0.999380,1.000000,0.999731,-0.976273,0.629509,1.000000,0.758091,0.981941
2,-0.006347,0.055216,0.003817,0.042302,-0.047071,-0.123372,-0.007745,0.083186
3,0.010455,-0.102704,-0.064634,-0.035412,-0.054411,-0.069507,-0.124977,-0.020064
4,0.969294,-0.999994,-0.997868,-0.999774,-0.981749,0.869530,-1.000000,-0.992149


In [31]:
diff = pd.DataFrame(predictions - mat)

In [32]:
diff.head()

,0,1,2,3,4,5,6,7
0,-0.023144,0.027928,0.232176,-0.261370,-0.412730,0.147745,0.252314,-0.403736
1,-3.410620,-14.830564,-6.294710,2.955673,-0.951165,-14.990311,-3.419680,-0.274383
2,-0.003435,-0.015272,0.013319,-0.039011,-0.034989,-0.032276,0.039684,0.018629
3,-0.024630,-0.040210,0.000708,-0.037466,-0.017533,0.000906,-0.017178,-0.004102
4,-7.026315,2.547690,1.724740,3.061579,9.137291,-8.261363,9.615977,12.870764


In [33]:
res['Euclidean_Distance'] = diff.apply(lambda series: series ** 2).apply(sum, axis = 1)

In [34]:
res['BBLE'] = df['BBLE']

In [35]:
res.head()

,0,1,2,3,4,5,6,7,Euclidean_Distance,BBLE
0,-0.023144,0.027928,0.232176,-0.261370,-0.412730,0.147745,0.252314,-0.403736,0.542375,1000010101
1,-3.410620,-14.830564,-6.294710,2.955673,-0.951165,-14.990311,-3.419680,-0.274383,517.320973,1000010201
2,-0.003435,-0.015272,0.013319,-0.039011,-0.034989,-0.032276,0.039684,0.018629,0.006132,1000020001
3,-0.024630,-0.040210,0.000708,-0.037466,-0.017533,0.000906,-0.017178,-0.004102,0.004248,1000020023
4,-7.026315,2.547690,1.724740,3.061579,9.137291,-8.261363,9.615977,12.870764,478.071607,1000030001


In [37]:
res.sort_values('Euclidean_Distance', ascending = False).head(10)

,0,1,2,3,4,5,6,7,Euclidean_Distance,BBLE
632815,-521.067444,354.248871,-749.743408,-274.291382,15.428579,100.409462,-51.101677,-38.997105,1.048807e+06,4018420001
776305,-136.843872,-620.757263,-22.366476,-488.043549,29.776876,38.818298,372.456268,-214.634628,8.299378e+05,4080100001
565391,-639.008484,-260.660309,372.454102,-63.076519,-38.287079,299.187714,-140.448364,229.638168,7.824150e+05,3085900700
1067359,-74.963837,-414.031158,-223.457748,136.410812,-8.310066,-456.603027,-421.253510,114.799744,6.447715e+05,5078530085
585117,-138.950836,53.675770,134.790955,102.026360,-440.511261,-14.745822,-90.501312,-240.259460,3.109491e+05,4004200001
585438,-120.424782,52.298180,95.811829,95.723137,-400.652374,-127.613327,91.992729,-167.753159,2.489913e+05,4004590005
565397,-285.719604,103.696220,204.333969,132.187134,-32.431759,92.849136,-222.412170,178.577454,2.426443e+05,3085910100
85885,-165.695282,80.684113,61.628704,92.601418,216.063309,-193.294724,211.424377,237.572586,2.315252e+05,1012540010
917941,-100.957306,27.505884,96.203049,51.693394,89.062195,92.434013,-230.519714,-292.688690,1.781583e+05,4142600001
750815,-19.205643,-108.692184,-52.073128,-38.614174,-7.785172,-301.918213,-235.299194,36.266518,1.642817e+05,4066610005E
